In [1]:
import torch
from model import DeepFakeDetector  # Import the model class

# Initialize and save the model
model = DeepFakeDetector()
torch.save(model.state_dict(), "deepfake_detector.pth")


IMPORTING AMD USING THE MODEL

In [ ]:
import torch
import cv2
import numpy as np
from facenet_pytorch import MTCNN
from moviepy.video.io.VideoFileClip import VideoFileClip

from model import DeepFakeDetector  # Import your model

# Initialize model and face detector
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = DeepFakeDetector().to(device)
model.load_state_dict(torch.load("deepfake_detector.pth", map_location=device))
model.eval()

mtcnn = MTCNN(keep_all=False, device=device)  # Face detector

# Function to preprocess an image
def preprocess_frame(frame):
    face = mtcnn(frame)
    if face is not None:
        face = face.permute(1, 2, 0).cpu().numpy()  # Convert tensor to numpy
        face = cv2.resize(face, (64, 64))  # Resize to match model input
        face = np.transpose(face, (2, 0, 1))  # Convert to (C, H, W)
        face = torch.tensor(face, dtype=torch.float32).unsqueeze(0).to(device)  # Add batch dim
        return face
    return None

# Process video
video_path = r"deepfake_test_videos\real\id5_0006.mp4"
clip = VideoFileClip(video_path)


for frame in clip.iter_frames(fps=5):  # Process every 5 frames per second
    face = preprocess_frame(frame)
    if face is not None:
        with torch.no_grad():
            output = model(face)  # Get prediction
        print(f"Prediction: {output.item()}")  # Adjust based on model output


{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf58.3.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [944, 500], 'bitrate': 1580, 'fps': 30.0, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 17.8, 'bitrate': 1582, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [944, 500], 'video_bitrate': 1580, 'video_fps': 30.0, 'video_duration': 17.8, 'video_n_frames': 534}
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe -i C:\ANU\AA AIML 6 Sem\Deep Learning Lab\video_dataset\Celeb-synthesi

TESTING THE MODEL FOR A SAMPLE VIDEO

In [20]:
predictions = []  # Store predictions

for frame in clip.iter_frames(fps=5):  # Process every 5 frames per second
    face = preprocess_frame(frame)
    if face is not None:
        with torch.no_grad():
            output = model(face)  # Get prediction (logit)
            prob = torch.sigmoid(output).item()  # Convert logit to probability
            predictions.append(prob)  # Store probability

# Compute final result
if predictions:
    avg_prob = np.mean(predictions)  # Average probability
    print(f"\nAverage Probability: {avg_prob:.4f}")

    threshold = 0.7  # Adjust if necessary
    result = "FAKE" if avg_prob > threshold else "REAL"
    print(f"The video is likely: {result} ")
else:
    print("No valid faces detected in the video! ")

{'video_found': True, 'audio_found': False, 'metadata': {'major_brand': 'isom', 'minor_version': '512', 'compatible_brands': 'isomiso2mp41', 'encoder': 'Lavf58.3.100'}, 'inputs': [{'streams': [{'input_number': 0, 'stream_number': 0, 'stream_type': 'video', 'language': None, 'default': True, 'size': [944, 500], 'bitrate': 1580, 'fps': 30.0, 'codec_name': 'mpeg4', 'profile': '(Simple Profile)', 'metadata': {'Metadata': '', 'handler_name': 'VideoHandler', 'vendor_id': '[0][0][0][0]'}}], 'input_number': 0}], 'duration': 17.8, 'bitrate': 1582, 'start': 0.0, 'default_video_input_number': 0, 'default_video_stream_number': 0, 'video_codec_name': 'mpeg4', 'video_profile': '(Simple Profile)', 'video_size': [944, 500], 'video_bitrate': 1580, 'video_fps': 30.0, 'video_duration': 17.8, 'video_n_frames': 534}
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\imageio_ffmpeg\binaries\ffmpeg-win-x86_64-v7.1.exe -i C:\ANU\AA AIML 6 Sem\Deep Learning Lab\video_dataset\Celeb-synthesi